In [2]:
import sys 
sys.path.append('/home/chendian/BBCM/')
from bbcm.data.loaders.confusor import *
conf = Confusor(cos_threshold=(0.1, 0.5), method='beam', token_sample_mode='sort', 
                pinyin_sample_mode='special', weight=[1, 0.3, 1], debug=False)

Use beam method.
Pinyin sampling mode: special.
Token sampling mode: sort.
Now loading pinyin2token corpus.
Now loading zi_sim_matrix.
Now loading REDscore.
Now generating score matrix.
Load word freuency data.


In [8]:
tw = '债券'
conf.method = ''
for w in range(10):
    conf.weight = [1, 0.1 * w, 1]
    confusion_set = conf(tw)
    print(f"{w}e-1", [f"{c}:{sum([c[i]!=_w for i, _w in enumerate(tw)])}" for c in confusion_set])

0e-1 ['宅圈:2', '债全:1', '宰圈:2', '在全:2', '在泉:2', '再全:2', '仔犬:2', '再圈:2', '栽拳:2', '再权:2']
1e-1 ['债全:1', '宅圈:2', '宰圈:2', '在全:2', '载权:2', '再圈:2', '载铨:2', '赵权:2', '再权:2', '仔犬:2']
2e-1 ['债全:1', '宅圈:2', '在全:2', '宰圈:2', '赵权:2', '载权:2', '再圈:2', '载铨:2', '债钱:1', '再权:2']
3e-1 ['债全:1', '宅圈:2', '在全:2', '赵权:2', '宰圈:2', '债钱:1', '载权:2', '再圈:2', '载铨:2', '债欠:1']
4e-1 ['债全:1', '宅圈:2', '在全:2', '赵权:2', '债钱:1', '宰圈:2', '载权:2', '债欠:1', '再圈:2', '载铨:2']
5e-1 ['债全:1', '在全:2', '赵权:2', '债钱:1', '宅圈:2', '债算:1', '债欠:1', '宰圈:2', '载权:2', '再圈:2']
6e-1 ['债全:1', '在全:2', '债钱:1', '赵权:2', '债算:1', '宅圈:2', '债欠:1', '宰圈:2', '载权:2', '再圈:2']
7e-1 ['债全:1', '债钱:1', '在全:2', '赵权:2', '债算:1', '债欠:1', '宅圈:2', '载权:2', '宰圈:2', '再圈:2']
8e-1 ['债全:1', '债钱:1', '在全:2', '赵权:2', '债算:1', '债欠:1', '载权:2', '窄缘:2', '宅圈:2', '宰圈:2']
9e-1 ['债全:1', '债钱:1', '在全:2', '赵权:2', '债算:1', '债欠:1', '窄缘:2', '载权:2', '宰圈:2', '宅院:2']


In [3]:
from bbcm.data.datasets.csc import PureTextDataset
fp = "/data/chendian/clean_pretrain_data_test"
dataset = PureTextDataset(fp)

Init indexing ends in 0.0005736351013183594 seconds
Loaded 958 files from /data/chendian/clean_pretrain_data_test.


In [4]:
from tools.train_csc import DynamicDataCollatorForCsc, BertTokenizer, get_abs_path

from bbcm.config import cfg
config_file = "fin/b_findoc_pret_test.yml"
if os.path.exists(config_file):
    cfg.merge_from_file(config_file)
else:
    cfg.merge_from_file(get_abs_path('configs', config_file))
tokenizer = BertTokenizer.from_pretrained(cfg.MODEL.BERT_CKPT)

tokenizer = BertTokenizer.from_pretrained(cfg.MODEL.BERT_CKPT)
col_fn_train = DynamicDataCollatorForCsc(tokenizer=tokenizer, augmentation=True)

Use beam method.
Pinyin sampling mode: special.
Token sampling mode: sort.
Now loading pinyin2token corpus.
Now loading zi_sim_matrix.
Now loading REDscore.
Now generating score matrix.
Load word freuency data.
Dynamic Data Collator Init
with 5.0% faulty positions.
Now loading NER model from /home/chendian/download/stanford-corenlp-4.2.2/


In [12]:
from bbcm.data.loaders.collator import *
from collections import Counter, defaultdict

changed_chars_in_word = defaultdict(int)
selected_word_length = defaultdict(int)
for i, sample in tqdm(enumerate(dataset)):
    ot, ct, wrong_id = sample
    o_text, c_text, current_wids, detail_dict = col_fn_train.sample_augment_single(
        ot, ct, wrong_id, valid_position_mask=True, 
        random_pos=True, word_level=True, detail=True)
    word_offsets = detail_dict['word_offsets']
    # inv_dict = {tuple(v): k for k, v in word_offsets.items()}
    for word_index, count in Counter([word_offsets.get(cw, [cw])[0] 
        for cw in current_wids]).most_common():
        changed_chars_in_word[count] += 1
        selected_word_length[len(word_offsets.get(word_index, [word_index]))] += 1


24588it [46:50, 10.14it/s]

Detected an invalid text: K 为特征因子，K=∑𝑖=1，k∈[-0.25,0.25]。
Failed for counting words
{(14,), (15,), (0,), (25, 26, 27, 28), (5, 6), (2,), (29,), (16, 17, 18, 19, 20, 21, 22, 23), (30,), (7,), (13,), (24,), (3, 4), (8, 9, 10, 11, 12)}
[0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [30] 29


24622it [46:54,  7.50it/s]

Detected an invalid text: 险最低资本计算公式𝑀𝐶人身险利率=Max[（AA 基础情景– PV 基础情景)－（AA 不利情景–PV 不利情景)，0]可知，当保持资产和负债的高度匹配时，资产和负债的利率敏感性才会高度一致，𝑀𝐶人身险利率才会降低。
Failed for counting words
{(111,), (54, 55), (59,), (74, 75), (86, 87), (5, 6), (23,), (36, 37), (61,), (97,), (46, 47), (98, 99, 100, 101), (95, 96), (78,), (16, 17), (80, 81), (64,), (91,), (1,), (76, 77), (19, 20, 21), (41,), (102, 103, 104), (56, 57), (70,), (108,), (3, 4), (18,), (27, 28), (58,), (85,), (22,), (60,), (29, 30), (68, 69), (50, 51, 52), (93, 94), (73,), (9, 10), (88, 89, 90), (105, 106), (62, 63), (66, 67), (11, 12), (43, 44), (83, 84), (33, 34), (48, 49), (79,), (0,), (38, 39), (65,), (92,), (109, 110), (2,), (40,), (24, 25), (31,), (42,), (107,), (71, 72), (82,), (13, 14, 15), (7, 8)}
[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,

61080it [1:51:52,  9.10it/s]


ValueError: Invalid index 958 with offset 61080

In [13]:
changed_chars_in_word

defaultdict(int, {1: 84926, 2: 23491, 3: 2241, 4: 89})